In [2]:
"""
例：F：失火，概率0.001；B：邻居告诉你失火，但撒谎概率0.1
则P(F|B) = 0.0089,这个数字说明，当获得了额外信息（邻居的话，不一定很准，也是概率分布），
预测失火的准确度将大大提高（从0.001升到0.0089），这就是sense利用BAYES公式 感知定位的原理
1.连续的 变为卷积运算
2.熵：Entropy=Σ(−p×log(p))
entropy will decrease after the measurement update (sense) step 
and that entropy will increase after the movement step (move)
entropy represents the amount of uncertainty in a system.
The maximum uncertainty occurs when all positions have equal probabilities [0.2, 0.2, 0.2, 0.2, 0.2]
the minimum uncertainty occurs when [0,1,0,0,0]
Taking a measurement will decrease uncertainty and entropy,whereas the movement will increase uncertainty
3. Summary of localization
belief = probability
sense = product followed by normalization 乘法，当world与感知相符时，乘大数；不符时，乘小数
move = convolution or addition 加法，状态转换连续时就是 概率p和状态转换函数的卷积运算
"""

#Given the list motions=[1,1] which means the robot 
#moves right and then right again, compute the posterior 
#distribution if the robot first senses red, then moves 
#right one, then senses green, then moves right again, 
#starting with a uniform prior distribution.


p=[0.2, 0.2, 0.2, 0.2, 0.2]
world=['green', 'red', 'red', 'green', 'green'] # 实际上，路标会代替这里的‘red’、‘green’；
                                        # 而且是整个场景的观测，是一张图片，因此须将图片嵌入到模型中作为measument
# measurements = ['red', 'green']
measurements = ['red', 'red']
motions = [1,1]
pHit = 0.6
pMiss = 0.2
pExact = 0.8
pOvershoot = 0.1
pUndershoot = 0.1

def sense(p, Z):
    """ 
    sense 就是 Bayes' rule
    贝叶斯公式p(x|z)= p(z|x)p(x) / p(z) 其中p(x|z)为后验概率，measurement后的概率，p(x)为先验概率p，
    p(z|x)是measurement probability,即在x处看到z的概率，相符时比较大pHit，在'red'处看到'red'概率大，看到'green'概率小
    p(z) = sum( p(z|x)p(x) ) 为了归一化
    """
    q=[]
    for i in range(len(p)):
        hit = (Z == world[i])
        q.append(p[i] * (hit * pHit + (1-hit) * pMiss))
    s = sum(q)
    for i in range(len(q)):
        q[i] = q[i] / s
    return q

def move(p, U):
    # move 就是 全概率公式，Total Probability: p(xi t) = Σ p(xj t-1)p(xi | xj) 
    # p(xi | xj)为从位置xj 到 xi的概率 或 P(A) = Σ P(A|B)P(B) 这里求和即为卷积运算
    q = []
    for i in range(len(p)):
        s = pExact * p[(i-U) % len(p)]
        s = s + pOvershoot * p[(i-U-1) % len(p)]
        s = s + pUndershoot * p[(i-U+1) % len(p)]
        q.append(s)
    return q
#
def loop(q,Zs,Us):
    for i in range(len(Zs)):
        q = sense(q,Zs[i])
        q = move(q,Us[i])
    return q
p = loop(p,measurements,motions)
#
print(p)        


[0.07882352941176471, 0.07529411764705884, 0.22470588235294123, 0.4329411764705882, 0.18823529411764706]
